In [3]:
!pip install langchain chromadb faiss-cpu  tiktoken  langchain-community wikipedia langchain_huggingface

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Wikipedia Retriever

In [4]:
from langchain_community.retrievers import WikipediaRetriever

In [6]:
retriver = WikipediaRetriever(top_k_results=2, lang = "en")

In [10]:
query = "AI boom or boon"
docs = retriver.invoke(query)

In [11]:
for i, doc in enumerate(docs):
  print(f"Result {i+1}")
  print()
  print(f"Content: {doc.page_content}")

Result 1

Content: Artificial intelligence is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. Artificial intelligence (AI) has been used in applications throughout industry and academia. Within the field of Artificial Intelligence, there are multiple subfields. The subfield of Machine learning has been used for various scientific and commercial purposes including language translation, image recognition, decision-making, credit scoring, and e-commerce. In recent years, there have been massive advancements in the field of Generative Artificial Intelligence, which uses generative models to produce text, images, videos or other forms of data. This article describes applications of AI in different sectors. 


== Agriculture ==

In agriculture, AI has been proposed as a way for farmers to identify areas that need irrigation, fertilization, or pesticide treatme

# VectorStore Retreiver

In [13]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import  Document

In [14]:
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [15]:
embedding = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
vectorStore = Chroma.from_documents(
    documents = docs,
    embedding = embedding,
    collection_name="retrievers"
)

In [20]:
from re import search
# Converting vector store into retriever
retriever1 = vectorStore.as_retriever(search_kwargs = {"k" : 2})
query = "What is MMR ?"
result = retriever1.invoke(query)

In [21]:
for i, doc in enumerate(result):
  print(f"Result {i+1}")
  print()
  print(doc.page_content)

Result 1

MMR helps you get diverse results when doing similarity search.
Result 2

LangChain makes it easy to work with LLMs.


# Maximal Marginal Relevance (MMR)

In [25]:
from langchain_community.vectorstores import FAISS
vectorStore2 = FAISS.from_documents(
    documents = docs,
    embedding = embedding
)

In [35]:
retriever2 = vectorStore2.as_retriever(search_type = "mmr",
                                       search_kwargs = {"k" : 3, "lambda_mult" : 0.5})

In [36]:
query1 = "What is Langchain"
result = retriever2.invoke(query1)

In [37]:
for i, doc in enumerate(result):
  print(f"Result {i+1}")
  print()
  print(doc.page_content)

Result 1

LangChain supports Chroma, FAISS, Pinecone, and more.
Result 2

LangChain is used to build LLM based applications.
Result 3

Embeddings are vector representations of text.


# Multi-Query Retriever

In [38]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.retrievers.multi_query import MultiQueryRetriever

In [39]:
healthDocs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [41]:
llm = HuggingFaceEndpoint(
    repo_id = "Qwen/Qwen3-4B-Instruct-2507",
    task = "text-generation",
    temperature = 0.5
)

model = ChatHuggingFace(llm = llm)

In [42]:
vectorStore3 = FAISS.from_documents(
    documents = healthDocs,
    embedding = embedding
)

## Comparison Between Normal Retriever and Multi Query Retriever

In [43]:
similarityRetriever = vectorStore3.as_retriever(search_type = "similarity", search_kwargs = {"k" : 3})

In [48]:
from langchain.retrievers.multi_query import MultiQueryRetriever
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorStore3.as_retriever(search_kwargs={"k": 3}),
    llm=model
)


In [50]:
queryyy = "How to improve energy levels and maintain balance"
similarity_results = similarityRetriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [51]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Black holes bend spacetime and store immense gravitational energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Python balances readability with power, making it a popular system design language.
******************************************************************************************************************************************************

--- Result 1 ---
Python balances readability with power, making it a popular system design language.

--- Result 2 ---
Black holes bend spacetime and store immense gravitational energy.

--- Result 3 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 4 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.


# Contextual Compression Retriever

In [52]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [54]:
docsss = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [55]:
vecStore = FAISS.from_documents(
    documents = docsss,
    embedding = embedding
)
baseRetriever = vecStore.as_retriever(search_kwargs = {"k" : 2})
compressor = LLMChainExtractor.from_llm(model)

compressionRetriver = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = baseRetriever
)

In [56]:
querryyyyyy = "What is Photosynthesis"
compressedResult = compressionRetriver.invoke(querryyyyyy)

In [58]:
for i, doc in enumerate(compressedResult):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
